- In this assignment we will be creating a multi-layer network that will be able to learn what a healthy lung looks like vs a non healthy lung that has Pnuemonia. 

- first step: PARSE THE DATA VERYYYYY TIME CONSUMMING


In [1]:
from PIL import Image
from resizeimage import resizeimage
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
from matplotlib import pyplot as plt
from os import listdir

tf.enable_eager_execution()

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
normal_url_train = "/Users/benjaminadame/Downloads/chest_xray/train/NORMAL/"
pneumonia_url_train = "/Users/benjaminadame/Downloads/chest_xray/train/PNEUMONIA/"
normal_url_test = "/Users/benjaminadame/Downloads/chest_xray/test/NORMAL/"
pneumonia_url_test = "/Users/benjaminadame/Downloads/chest_xray/test/PNEUMONIA/"
input_size = 2500

#initializing training and testing matrices 
def initialize_test_train(n,p):
    #size of the image that we want
    size = [50,50]
    i = 0
    #Open trainng directorys
    n_d = listdir(n)
    p_d = listdir(p)
    bug = ".DS_Store"
      
    #Removing DS_store from directory name:::
    if bug in n_d:
        n_d.remove(bug)
    
    if bug in p_d:
        p_d.remove(bug)
        
    #nnp: number of normal pictures in training data
    nnp = len(n_d)
    #npp: number of pneumonia pictures in training data
    npp = len(p_d)
    #row is speaking of image size which is 50*50*3
    x = np.zeros((input_size,nnp+npp))
    #row is speaking about label 1 label for each
    y = np.zeros((1,nnp+npp))
    #Process NORMAL lung files for training and testing set 
    for file in n_d:
        #print(file)    
        #Resizing images
        with open(n + file, 'r+b') as f:
            with Image.open(f) as image:
                cover = resizeimage.resize_cover(image, size)
                cover = np.array(cover)
                if cover.shape == (50, 50):
                    cover = cover.reshape(input_size,)
                    x[:,i] = cover
                    y[:,i] = 0
                    i += 1
            
    #Process PNEUMONIA lung files for training and testing set 
    for file in p_d:
        #print(file)
        #Resize images
        with open(p + file, 'r+b') as f:
            with Image.open(f) as image:
                cover = resizeimage.resize_cover(image, size)
                cover = np.array(cover)
                if cover.shape == (50,50):
                    cover = cover.reshape(input_size,)
                    x[:, i] = cover
                    y[:, i] = 1
                    i += 1
    return x[:, 0:i], y[:, 0:i]
    
#call x and y to initialize matrices
X_train, Y_train = initialize_test_train(normal_url_train,pneumonia_url_train)
X_test, Y_test = initialize_test_train(normal_url_test,pneumonia_url_test)

#print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)


In [17]:
#Tensorflow Datasets
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
Y_train = tf.convert_to_tensor(Y_train, dtype=tf.float32)
Y_test = tf.convert_to_tensor(Y_test, dtype=tf.float32)

#create train dataset
train_dataset = tf.data.Dataset.from_tensors((X_train,Y_train))
test_dataset = tf.data.Dataset.from_tensors((X_test,Y_test))



Construct the Network Architechture

In [18]:
#model architechture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(5,activation = "relu",input_shape = (2500,)),
    tf.keras.layers.Dense(3,activation = "relu"),
    tf.keras.layers.Dense(2,activation = "relu"),
    tf.keras.layers.Dense(1,activation = "sigmoid")
])

Construct the foward propagation function 

In [19]:
def forward_prop(model,x,y):
    a = model(x)
    loss = tf.losses.sigmoid_cross_entropy(y,a)
    return loss

Get Gradienr Descent //optimizer 


In [20]:
lr = .1
optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)


Training Time!!!

In [21]:
m = 5232
epochs = 1000

#loop it
for epoch in range(epochs):
    loss_avg = tfe.metrics.Mean()
    
    for x,y in train_dataset:
        with tf.GradientTape(persistent = True) as tape:
            loss = forward_prop(model,x,y)
            #grad: gives you dw,db while model.vars gives you w and b
        grad = tape.gradient(loss,model.variables)
        loss_avg(loss)
        #update with grad descent 
        optimizer.apply_gradients(zip(grad,model.variables))
        
    if epoch % 25 == 0:
        print("cost: {}".format(loss_avg.result()))

InvalidArgumentError: cannot compute MatMul as input #0 was expected to be a float tensor but is a double tensor [Op:MatMul]